# SISTEM INFORMASI DESA - KELURAHAN DASAN GERES KEC. GERUNG KAB. LOMBOK BARAT

In [2]:
import pandas as pd
import numpy as np
import os
import re

## CLEANING

### Konfigurasi Kolom

In [30]:
# KONFIGURASI KOLOM TARGET (41 KOLOM STANDAR SID)
target_cols = [
    'alamat', 'dusun', 'rw', 'rt', 'nama', 'no_kk', 'nik', 'sex', 'tempatlahir', 
    'tanggallahir', 'agama_id', 'pendidikan_kk_id', 'pendidikan_sedang_id', 
    'pekerjaan_id', 'status_kawin', 'kk_level', 'warganegara_id', 'ayah_nik', 
    'nama_ayah', 'ibu_nik', 'nama_ibu', 'golongan_darah_id', 'akta_lahir', 
    'dokumen_pasport', 'tanggal_akhir_paspor', 'dokumen_kitas', 'akta_perkawinan', 
    'tanggalperkawinan', 'akta_perceraian', 'tanggalperceraian', 'cacat_id', 
    'cara_kb_id', 'hamil', 'ktp_el', 'status_rekam', 'alamat_sekarang', 
    'status_dasar', 'suku', 'tag_id_card', 'id_asuransi', 'no_asuransi'
]

map_pekerjaan = {
    'BELUM/TIDAK BEKERJA': 1, 'MENGURUS RUMAH TANGGA': 2, 'PELAJAR/MAHASISWA': 3,
    'PENSIUNAN': 4, 'PEGAWAI NEGERI SIPIL (PNS)': 5, 'TNI': 6, 'POLRI': 7,
    'PERDAGANGAN': 8, 'PETANI/PEKEBUN': 9, 'PETERNAK': 10, 'NELAYAN/PERIKANAN': 11,
    'INDUSTRI': 12, 'KONSTRUKSI': 13, 'TRANSPORTASI': 14, 'KARYAWAN SWASTA': 15,
    'KARYAWAN BUMN': 16, 'KARYAWAN BUMD': 17, 'KARYAWAN HONORER': 18,
    'BURUH HARIAN LEPAS': 19, 'BURUH TANI/PERKEBUNAN': 20, 'BURUH NELAYAN/PERIKANAN': 21,
    'BURUH PETERNAKAN': 22, 'PEMBANTU RUMAH TANGGA': 23, 'TUKANG CUKUR': 24,
    'TUKANG LISTRIK': 25, 'TUKANG BATU': 26, 'TUKANG KAYU': 27, 'TUKANG SOL SEPATU': 28,
    'TUKANG LAS/PANDAI BESI': 29, 'TUKANG JAHIT': 30, 'PENATA RAMBUT': 31,
    'PENATA RIAS': 32, 'PENATA BUSANA': 33, 'MEKANIK': 34, 'TUKANG GIGI': 35,
    'SENIMAN': 36, 'TABIB': 37, 'PARAJI': 38, 'PERANCANG BUSANA': 39,
    'PENTERJEMAH': 40, 'IMAM MASJID': 41, 'PENDETA': 42, 'PASTOR': 43,
    'WARTAWAN': 44, 'USTADZ/MUBALIGH': 45, 'JURU MASAK': 46, 'PROMOTOR ACARA': 47,
    'ANGGOTA DPR-RI': 48, 'ANGGOTA DPD': 49, 'ANGGOTA BPK': 50, 'PRESIDEN': 51,
    'WAKIL PRESIDEN': 52, 'ANGGOTA MAHKAMAH KONSTITUSI': 53, 'ANGGOTA KABINET/MENTERI': 54,
    'DUTA BESAR': 55, 'GUBERNUR': 56, 'WAKIL GUBERNUR': 57, 'BUPATI': 58,
    'WAKIL BUPATI': 59, 'WALIKOTA': 60, 'WAKIL WALIKOTA': 61, 'ANGGOTA DPRD PROVINSI': 62,
    'ANGGOTA DPRD KABUPATEN/KOTA': 63, 'DOSEN': 64, 'GURU': 65, 'PILOT': 66,
    'PENGACARA': 67, 'NOTARIS': 68, 'ARSITEK': 69, 'AKUNTAN': 70, 'KONSULTAN': 71,
    'DOKTER': 72, 'BIDAN': 73, 'PERAWAT': 74, 'APOTEKER': 75, 'PSIKIATER/PSIKOLOG': 76,
    'PENYIAR TELEVISI': 77, 'PENYIAR RADIO': 78, 'PELAUT': 79, 'PENELITI': 80,
    'SOPIR': 81, 'PIALANG': 82, 'PARANORMAL': 83, 'PEDAGANG': 84, 'PERANGKAT DESA': 85,
    'KEPALA DESA': 86, 'BIARAWAN': 87, 'WIRASWASTA': 88, 'ANGGOTA LEMBAGA TINGGI': 89,
    'ARTIS': 90, 'ATLET': 91, 'MANAJER': 92, 'TENAGA KEPENDIDIKAN': 93,
    'TENAGA KESEHATAN': 94, 'ASISTEN TENAGA KESEHATAN': 95, 'TENAGA TEKNIK KEFARMASIAN': 96,
    'TEKNISI MEDIS': 97, 'TENAGA KETEKNISIAN MEDIS': 98, 'LAINNYA': 99
}

map_sex = {'L': 1, 'P': 2, 'LAKI-LAKI': 1, 'PEREMPUAN': 2}

map_agama = {'ISLAM': 1, 'KRISTEN': 2, 'KATHOLIK': 3, 'HINDU': 4, 'BUDHA': 5, 'KHONGHUCU': 6}

map_pendidikan_kk = {'TIDAK / BELUM SEKOLAH': 1, 'BELUM TAMAT SD/SEDERAJAT': 2, 'TAMAT SD/SEDERAJAT': 3,
                     'SLTP/SEDERAJAT': 4, 'SLTA/SEDERAJAT': 5, 'DIPLOMA I/II': 6, 'AKADEMI/DIPLOMA III/SARJANA MUDA': 7,
                     'DIPLOMA IV/STRATA I': 8, 'STRATA II': 9, 'STRATA III': 10}

map_pendidikan_sedang = {
    'TIDAK / BELUM SEKOLAH': 1, 'BELUM TAMAT SD / SEDERAJAT': 2, 'TAMAT SD / SEDERAJAT': 3,
    'SLTP / SEDERAJAT': 4, 'SLTA / SEDERAJAT': 5, 'DIPLOMA I / II': 6,
    'AKADEMI / DIPLOMA III / S. MUDA': 7, 'DIPLOMA IV / STRATA I': 8, 'STRATA II': 9,
    'STRATA III': 10, 'PAUD': 11, 'TK': 12, 'LAINNYA': 13}

map_kawin = {'BELUM KAWIN': 1, 'KAWIN': 2, 'CERAI HIDUP': 3, 'CERAI MATI': 4}

map_kk_level = {'KEPALA KELUARGA': 1, 'KK': 1, 'SUAMI': 2, 'ISTRI': 3, 
                'ANAK': 4, 'MENANTU': 5, 'CUCU': 6, 'ORANG TUA': 7, 
                'MERTUA': 8, 'FAMILI LAIN': 9, 'PEMBANTU': 10, 'LAINNYA': 11}

map_warganegara = {'WNI': 1, 'WNA': 2, 'DUA KEWARGANEGARAAN': 3}

map_status_dasar = {'HIDUP': 1, 'MENINGGAL': 2, 'PINDAH': 3, 'HILANG': 4, 'PERGI': 5, 'TIDAK VALID': 6}

map_hubungan = {'KEPALA KELUARGA': 1, 'KK': 1, 'SUAMI': 2, 'ISTRI': 3, 'ANAK': 4, 'CUCU': 6}

map_golongan_darah = {
    'A': 1, 'B': 2, 'AB': 3, 'O': 4, 
    'A+': 5, 'A-': 6, 'B+': 7, 'B-': 8, 
    'AB+': 9, 'AB-': 10, 'O+': 11, 'O-': 12, 
    'TIDAK TAHU': 13, '-': 13
}

map_jamkesnas = {
    'TIDAK/BELUM PUNYA': 1,
    'BPJS PBI': 2, # Penerima Bantuan Iuran
    'BPJS NON PBI': 3,
    'BPJS DAERAH': 4,
    'ASURANSI LAINNYA': 99
}

map_cacat = {
    'CACAT FISIK': 1,
    'CACAT NETRA/BUTA': 2,
    'CACAT RUNGU/WICARA': 3,
    'CACAT MENTAL/JIWA': 4,
    'CACAT FISIK DAN MENTAL': 5,
    'CACAT LAINNYA': 6,
    'TIDAK CACAT': 7
}

map_cara_kb = {
    'PIL': 1, 'IUD': 2, 'SUNTIK': 3, 'KONDOM': 4, 
    'SUSUK KB': 5, 'STERILISASI WANITA': 6, 'STERILISASI PRIA': 7, 
    'TIDAK KB / LAINNYA': 99
}

map_hamil = {'YA': 1, 'TIDAK': 2}

map_ktp_el = {'BELUM': 1, 'SUDAH': 2}


In [31]:
def clean_and_map(val, mapping, default=99):
    if pd.isna(val) or val == "" or val == "-": return default
    v = str(val).strip().upper()
    for key in mapping:
        if key in v:
            return mapping[key]
    return default

def fix_0f_format(val):
    if pd.isna(val) or val == '':
        return ''
    # Jika tipenya float (angka yang ada .0 nya), format jadi angka bulat
    if isinstance(val, float):
        return '{:.0f}'.format(val)
    # Jika tipenya sudah string, bersihkan spasi dan .0 di ujung
    val_str = str(val).strip().replace(' ', '')
    if val_str.endswith('.0'):
        val_str = val_str[:-2]
    return val_str

### Formatting

In [32]:
output_folder = 'clean'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    print(f"📁 Folder '{output_folder}' berhasil dibuat.")

def transform(file_path):
    print(f"📖 Membuka file: {file_path}")
    
    xls = pd.ExcelFile(file_path)
    all_rt_data = []

    for sheet in xls.sheet_names:
        if "RT" not in sheet.upper(): continue
        print(f"   ∟ Memproses {sheet}...")
        
        # skiprows=4 karena header utama ada di baris 1-4
        df = pd.read_excel(xls, sheet_name=sheet, skiprows=4)
        if df.empty or len(df.columns) < 10: continue
        
        # Melewati sub-header (Tempat, Tanggal, Tahun) yang biasanya ada di baris pertama data
        df_clean = df.iloc[2:].reset_index(drop=True)
        
        # Forward Fill No. KK (Kolom index 2) - Mengatasi sel kosong di bawah KK
        df_clean.iloc[:, 2] = df_clean.iloc[:, 2].replace(0, np.nan).ffill()

        new_df = pd.DataFrame(index=df_clean.index, columns=target_cols)
        
        header_df = pd.read_excel(xls, sheet_name=sheet, nrows=5, header=None)
        raw_dusun = str(header_df.iloc[2, 1]) # Biasanya di baris 3 (index 2), kolom B (index 1)
        
        # Regex untuk hapus "Lingkungan" atau kata pertama (case insensitive)
        clean_dusun = re.sub(r'^(lingkungan|dusun)\s+', '', raw_dusun, flags=re.IGNORECASE).strip().upper()
        
        # 1. Mapping Identitas (Menggunakan index kolom agar stabil)
        new_df['nama'] = df_clean.iloc[:, 4].astype(str).str.upper().str.strip()
        new_df['no_kk'] = df_clean.iloc[:, 2].astype(str).str.replace(".0", "", regex=False).str.replace(" ", "")
        new_df['nik'] = df_clean.iloc[:, 6].astype(str).str.replace(".0", "", regex=False).str.replace(" ", "")
        new_df['sex'] = df_clean.iloc[:, 7].astype(str).str.upper().str.strip()
        new_df['status_kawin'] = df_clean.iloc[:, 8].astype(str).str.upper().str.strip()
        new_df['dusun'] = clean_dusun
        new_df['rt'] = sheet.replace("RT", "").strip()
        new_df['tempatlahir'] = df_clean.iloc[:, 9].astype(str).str.upper()
        new_df['suku'] = df_clean.iloc[:, 14].astype(str).str.upper().str.strip()
        new_df['pekerjaan'] = df_clean.iloc[:, 17].astype(str).str.upper().str.strip()
        new_df['agama'] = df_clean.iloc[:, 13].astype(str).str.upper().str.strip()
        new_df['kk_level'] = df_clean.iloc[:, 5].astype(str).str.upper().str.strip()
        new_df['pendidikan_kk'] = df_clean.iloc[:, 15].astype(str).str.upper().str.strip()

        # 2. Transformasi Tanggal 
        raw_dates = df_clean.iloc[:, 10]
        formatted_dates = pd.to_datetime(raw_dates, errors='coerce')
        new_df['tanggallahir'] = formatted_dates.dt.strftime('%Y-%m-%d')
        
        # 3. Mapping ID Numerik
        new_df['sex_id'] = df_clean.iloc[:, 7].apply(lambda x: clean_and_map(x, map_sex, 1))
        new_df['status_kawin_id'] = df_clean.iloc[:, 8].apply(lambda x: clean_and_map(x, map_kawin, 1))
        new_df['pekerjaan_id'] = df_clean.iloc[:, 17].apply(lambda x: clean_and_map(x, map_pekerjaan, 99))
        new_df['agama_id'] = df_clean.iloc[:, 13].apply(lambda x: clean_and_map(x, map_agama, 1))
        new_df['kk_level_id'] = df_clean.iloc[:, 5].apply(lambda x: clean_and_map(x, map_kk_level, 1))
        new_df['pendidikan_kk_id'] = df_clean.iloc[:, 15].apply(lambda x: clean_and_map(x, map_pendidikan_kk, 1))
        
        # 4. Default Values
        # new_df[['status_dasar', 'warganegara_id']] = 1
        # new_df[['cacat_id', 'hamil']] = [7, 2]
        # new_df['suku'] = 'SASAK'
        
        all_rt_data.append(new_df)

    if all_rt_data:
        final_df = pd.concat(all_rt_data, ignore_index=True)
        # Hapus baris yang namanya kosong/nan
        return final_df[final_df['nama'].notna() & (final_df['nama'] != 'NAN')]
    else:
        return pd.DataFrame(columns=target_cols)

#### Dasan Geres Barat

In [33]:
# --- DASAN GERES BARAT ---
path_barat = 'dataset/dasan-geres-barat.xlsx'
if os.path.exists(path_barat):
    df_dasan_geres_barat = transform(path_barat)
    print(f"\n✅ Selesai! Berhasil memproses {len(df_dasan_geres_barat)} jiwa di Dusun Dasan Geres Barat.")
    
    save_path = os.path.join(output_folder, 'DASAN_GERES_BARAT.csv')
    df_dasan_geres_barat.to_csv(save_path, index=False)
    print(f"💾 File berhasil disimpan di: {save_path}")
    
else:
    print(f"❌ File {path_barat} tidak ditemukan!")

📖 Membuka file: dataset/dasan-geres-barat.xlsx
   ∟ Memproses RT 01...
   ∟ Memproses RT 02...
   ∟ Memproses RT 03...
   ∟ Memproses RT 04...
   ∟ Memproses RT 05...

✅ Selesai! Berhasil memproses 1111 jiwa di Dusun Dasan Geres Barat.
💾 File berhasil disimpan di: clean\DASAN_GERES_BARAT.csv


In [34]:
clean_dasan_geres_barat = pd.read_csv('clean/DASAN_GERES_BARAT.csv')
clean_dasan_geres_barat['nik'] = clean_dasan_geres_barat['nik'].apply(fix_0f_format)
clean_dasan_geres_barat['no_kk'] = clean_dasan_geres_barat['no_kk'].apply(fix_0f_format)
print("Sampel Data:")
clean_dasan_geres_barat[['nama', 'sex', 'no_kk', 'nik', 'tanggallahir', 'dusun', 'pekerjaan_id', 'suku', 'agama_id', 'pendidikan_kk_id']].sample(10)

Sampel Data:


,nama,sex,no_kk,nik,tanggallahir,dusun,pekerjaan_id,suku,agama_id,pendidikan_kk_id
788,MUH. SYUKRON,P,5201010503082293,520101200400018,1999-08-17,DASAN GERES BARAT,99,NAN,1,1
536,FATMI MALIHAM,P,5201010503084613,5201010909960002,1996-09-09,DASAN GERES BARAT,3,SASAK,1,5
358,ROHANIAH,P,5201010503084573,5201015303660001,1966-03-13,DASAN GERES BARAT,99,NAN,1,1
521,MUHAMMAD . AMRULLOH,L,5201010503082273,5201012304961001,1996-04-23,DASAN GERES BARAT,3,SASAK,1,4
7,MUHAMMAD NUR ALFARIDZI,L,5201011506120010,5201010302990003,1999-02-03,DASAN GERES BARAT,99,SASAK,1,1
166,MIZA LARADINI,P,5201010412131015,5201014504041001,2004-04-05,DASAN GERES BARAT,99,SASAK,1,1
262,HJ. NURHIDAYAH,P,52010111012100024,0,1953-07-01,DASAN GERES BARAT,84,NAN,1,1
323,HAERUNISA,P,5201011206150017,5271046106860001,1986-06-21,DASAN GERES BARAT,99,SASAK,1,1
310,M TAUHID S.P,L,5201010503082828,5201013009700002,1970-09-30,DASAN GERES BARAT,88,SASAK,1,1
692,ANDRI ARIFANI PRATAMA,L,5201011304110000,5201011205080003,2008-05-12,DASAN GERES BARAT,1,SASAK,1,1


#### Dasan Geres Timur

In [35]:
# --- DASAN GERES TIMUR ---
path_timur = 'dataset/dasan-geres-timur.xlsx'
if os.path.exists(path_timur):
    df_dasan_geres_timur = transform(path_timur)
    print(f"\n✅ Selesai! Berhasil memproses {len(df_dasan_geres_timur)} jiwa di Dusun Dasan Geres Timur.")

    save_path = os.path.join(output_folder, 'DASAN_GERES_TIMUR.csv')
    df_dasan_geres_timur.to_csv(save_path, index=False)
    print(f"💾 File berhasil disimpan di: {save_path}")
    
else:
    print(f"❌ File {path_timur} tidak ditemukan!")

📖 Membuka file: dataset/dasan-geres-timur.xlsx
   ∟ Memproses RT 01...
   ∟ Memproses RT 02...
   ∟ Memproses RT 03...
   ∟ Memproses RT 04...

✅ Selesai! Berhasil memproses 827 jiwa di Dusun Dasan Geres Timur.
💾 File berhasil disimpan di: clean\DASAN_GERES_TIMUR.csv


In [36]:
clean_dasan_geres_timur = pd.read_csv('clean/DASAN_GERES_TIMUR.csv')
clean_dasan_geres_timur['nik'] = clean_dasan_geres_timur['nik'].apply(fix_0f_format)
clean_dasan_geres_timur['no_kk'] = clean_dasan_geres_timur['no_kk'].apply(fix_0f_format)
print("Sampel Data:")
clean_dasan_geres_timur[['nama', 'sex', 'nik', 'no_kk', 'tanggallahir', 'dusun', 'pekerjaan_id', 'suku', 'agama_id','pendidikan_kk_id']].sample(10)

Sampel Data:


,nama,sex,nik,no_kk,tanggallahir,dusun,pekerjaan_id,suku,agama_id,pendidikan_kk_id
693,HAERUN,L,5201011303891001,5201010503086161,1989-03-13,DASAN GERES TIMUR,9,SASAK,1,4
235,MAHDI,L,5201013112940037,5201011807120014,1994-12-31,DASAN GERES TIMUR,99,SASAK,1,4
361,MUHAMAD FAHRI HAMZAH,L,5201010311140002,5201010503083374,2014-11-03,DASAN GERES TIMUR,99,SASAK,1,1
290,SUMIATI,P,5201014107550140,5201010503085193,1955-07-01,DASAN GERES TIMUR,99,SASAK,1,1
820,NUR ARTI LUSSY,P,5201014203930002,5201011606150001,1993-03-02,DASAN GERES TIMUR,74,SASAK,1,1
431,JAKNAH,P,5201014107860349,5201010503083014,1986-07-01,DASAN GERES TIMUR,1,SASAK,1,4
456,HAERIAH,P,5201014102641001,5201010503083104,1964-02-01,DASAN GERES TIMUR,99,SASAK,1,3
536,"NIKMAH, S.PD",P,5201016912850001,5201010703110108,1985-12-29,DASAN GERES TIMUR,65,SASAK,1,1
400,SAYADI,L,5201013112570013,5201011102100005,1957-12-31,DASAN GERES TIMUR,99,SASAK,1,5
534,ADAM ARESTU JAYADI,L,5201011105180001,5201012303180003,2018-05-11,DASAN GERES TIMUR,1,SASAK,1,1


In [37]:
# --- DASAN GERES SELATAN ---
path_selatan = 'dataset/dasan-geres-selatan.xlsx'
if os.path.exists(path_selatan):
    df_dasan_geres_selatan = transform(path_selatan)
    print(f"\n✅ Selesai! Berhasil memproses {len(df_dasan_geres_selatan)} jiwa di Dusun Dasan Geres Selatan.")

    save_path = os.path.join(output_folder, 'DASAN_GERES_SELATAN.csv')
    df_dasan_geres_selatan.to_csv(save_path, index=False)
    print(f"💾 File berhasil disimpan di: {save_path}")

else:
    print(f"❌ File {path_selatan} tidak ditemukan!")

    

📖 Membuka file: dataset/dasan-geres-selatan.xlsx
   ∟ Memproses RT 01...
   ∟ Memproses RT 02...
   ∟ Memproses RT 03...
   ∟ Memproses RT 04...

✅ Selesai! Berhasil memproses 766 jiwa di Dusun Dasan Geres Selatan.
💾 File berhasil disimpan di: clean\DASAN_GERES_SELATAN.csv


In [38]:
clean_dasan_geres_selatan = pd.read_csv('clean/DASAN_GERES_SELATAN.csv')
clean_dasan_geres_selatan['nik'] = clean_dasan_geres_selatan['nik'].apply(fix_0f_format)
clean_dasan_geres_selatan['no_kk'] = clean_dasan_geres_selatan['no_kk'].apply(fix_0f_format)
print("Sampel Data:")
clean_dasan_geres_selatan[['nama', 'sex_id', 'nik', 'no_kk', 'tanggallahir', 'dusun', 'pekerjaan_id', 'suku', 'agama_id', 'pendidikan_kk_id']].sample(10)

Sampel Data:


,nama,sex_id,nik,no_kk,tanggallahir,dusun,pekerjaan_id,suku,agama_id,pendidikan_kk_id
324,MUSTIANA MAULIDA,2,5201015601130003,5201010503084036,2013-01-16,DASN GERES SELATAN,1,SASAK,1,1
23,HAJJAH AZIZAH,2,5201017112850158,5201010503082568,1985-12-31,DASN GERES SELATAN,88,NAN,1,1
120,WAHYU TAUFIK HIDAYAT,1,5201011109000003,5201010503082417,2000-09-11,DASN GERES SELATAN,99,SASAK,1,2
31,M SALIM SUNGKAR,1,5201010107780001,5201010503084063,1978-05-15,DASN GERES SELATAN,81,SASAK,1,5
534,MAEMANAH,2,5201017112860112,,1986-12-31,DASN GERES SELATAN,88,NAN,1,1
320,KHAIRIL APRIANTO,1,5201011704130001,5201010503083066,2013-04-17,DASN GERES SELATAN,99,SASAK,1,1
538,HAJJAH MUKMINAH,2,0,,1942-12-31,DASN GERES SELATAN,99,NAN,1,1
21,MUHAMMAD DAFFA WAIZ,1,5201012102190002,5201010503082568,2019-02-21,DASN GERES SELATAN,1,SASAK,1,1
121,CINTA AULIA LESTARI,2,5201016906070001,5201010503082417,2007-06-29,DASN GERES SELATAN,1,SASAK,1,1
135,LALU ARYAPATI IBNU WAHID,1,5201012009960001,5201011107170009,1996-09-20,DASN GERES SELATAN,3,SASAK,1,5


#### Dasan Geres Tengah

In [39]:
# --- DASAN GERES TENGAH ---
path_tengah = 'dataset/dasan-geres-tengah.xlsx'
if os.path.exists(path_tengah):
    df_dasan_geres_tengah = transform(path_tengah)
    print(f"\n✅ Selesai! Berhasil memproses {len(df_dasan_geres_tengah)} jiwa di Dusun Dasan Geres Tengah.")

    save_path = os.path.join(output_folder, 'DASAN_GERES_TENGAH.csv')
    df_dasan_geres_tengah.to_csv(save_path, index=False)
    print(f"💾 File berhasil disimpan di: {save_path}")

else:
    print(f"❌ File {path_tengah} tidak ditemukan!")

📖 Membuka file: dataset/dasan-geres-tengah.xlsx
   ∟ Memproses RT 1...
   ∟ Memproses RT 2...
   ∟ Memproses RT 3...
   ∟ Memproses RT 4...

✅ Selesai! Berhasil memproses 883 jiwa di Dusun Dasan Geres Tengah.
💾 File berhasil disimpan di: clean\DASAN_GERES_TENGAH.csv


In [40]:
clean_dasan_geres_tengah = pd.read_csv('clean/DASAN_GERES_TENGAH.csv')
clean_dasan_geres_tengah['nik'] = clean_dasan_geres_tengah['nik'].apply(fix_0f_format)
clean_dasan_geres_tengah['no_kk'] = clean_dasan_geres_tengah['no_kk'].apply(fix_0f_format)
print("Sampel Data:")
clean_dasan_geres_tengah[['nama', 'sex_id', 'nik', 'no_kk', 'tanggallahir', 'dusun', 'pekerjaan_id', 'suku', 'agama_id', 'pendidikan_kk_id']].sample(10)

Sampel Data:


,nama,sex_id,nik,no_kk,tanggallahir,dusun,pekerjaan_id,suku,agama_id,pendidikan_kk_id
16,MUHAMAD ALWAN TAUFIQURAHMAN,1,5201010208080006,5201013001150023,2008-08-02,DASAN GERES TENGAH,1,SASAK,1,1
537,LALU SYARIF ABDILLAH,1,5201011509080001,5201010503082642,2008-09-15,DASAN GERES TENGAH,1,SASAK,1,1
394,ADNAN,1,5201011212630011,5201012112090037,1963-12-12,DASAN GERES TENGAH,20,SASAK,1,5
561,"NURLAELY ROMDAYANI, S. A.MD",2,5201016807810001,5201010909120066,1981-07-28,DASAN GERES TENGAH,5,SASAK,1,1
477,SAHARUDIN,1,5201010107640202,5201012402110025,1954-07-01,DASAN GERES TENGAH,19,SASAK,1,3
381,AMINAH,2,5201014107470217,5201010509160006,1945-01-07,DASAN GERES TENGAH,99,NAN,1,1
471,MAHDI,1,5201013112940037,5201011807120014,1994-12-31,DASAN GERES TENGAH,1,SASAK,1,4
360,MUHAJIRIN,1,5201012110940001,5201013012160001,1994-10-21,DASAN GERES TENGAH,88,SASAK,1,5
865,GHAZI USMAN YURBI,1,5201010107060243,5201010503082868,2007-04-18,DASAN GERES TENGAH,1,SASAK,1,1
344,ARSAN,1,5201010107520269,52010105030825704,1952-07-01,DASAN GERES TENGAH,19,SASAK,1,1


#### Aik Ampat

In [41]:
# --- AIK AMPAT ---
path_aik_ampat = 'dataset/aik-ampat.xlsx'
if os.path.exists(path_aik_ampat):
    df_aik_ampat = transform(path_aik_ampat)
    print(f"\n✅ Selesai! Berhasil memproses {len(df_aik_ampat)} jiwa di Dusun Aik Ampat.")

    save_path = os.path.join(output_folder, 'AIK_AMPAT.csv')
    df_aik_ampat.to_csv(save_path, index=False)
    print(f"💾 File berhasil disimpan di: {save_path}")

else:
    print(f"❌ File {path_aik_ampat} tidak ditemukan!")

📖 Membuka file: dataset/aik-ampat.xlsx
   ∟ Memproses RT 01...
   ∟ Memproses RT 02...
   ∟ Memproses RT 03...
   ∟ Memproses RT 04...

✅ Selesai! Berhasil memproses 1078 jiwa di Dusun Aik Ampat.
💾 File berhasil disimpan di: clean\AIK_AMPAT.csv


In [42]:
clean_aik_ampat = pd.read_csv('clean/AIK_AMPAT.csv')
clean_aik_ampat['nik'] = clean_aik_ampat['nik'].apply(fix_0f_format)
clean_aik_ampat['no_kk'] = clean_aik_ampat['no_kk'].apply(fix_0f_format)
print("Sampel Data:")
clean_aik_ampat[['nama', 'sex_id', 'nik', 'no_kk', 'tanggallahir', 'dusun', 'pekerjaan_id', 'suku', 'agama_id', 'pendidikan_kk_id']].sample(10)

Sampel Data:


,nama,sex_id,nik,no_kk,tanggallahir,dusun,pekerjaan_id,suku,agama_id,pendidikan_kk_id
894,MUNAIRI,2,5201016210810001,520101050308575,1981-10-22,AIK AMPAT,99,NAN,1,1
76,MUGNI,1,5201015310880002,5201010602150013,1988-12-31,AIK AMPAT,88,SASAK,1,3
892,MUHAMAD ROMI,1,5201010507880004,520101050308575,1994-04-25,AIK AMPAT,88,SASAK,1,5
483,M DIKY SAPUTRA,1,5201012602140002,5201010503089335,2004-02-26,AIK AMPAT,99,SASAK,1,1
325,NURUL SA'ADATUL UMMAH,2,5201016912150001,5201011212120011,2015-12-29,AIK AMPAT,99,SASAK,1,1
98,ASYAM WADZIMMIY,1,5201010602130002,5201012305120020,2013-02-06,AIK AMPAT,99,SASAK,1,1
818,PARA HARDIANTI,2,0,5201010503085113,2012-08-15,AIK AMPAT,99,NAN,1,1
49,NURISAH,2,5201014107680328,5201010212140007,1968-07-01,AIK AMPAT,19,SASAK,1,1
643,MUHAMAD ALI ADWAN,1,5201010912410003,5201010503085003,2014-12-09,AIK AMPAT,99,SASAK,1,1
502,SITI PAHARNI,2,5201015011000001,5201010503084964,1998-12-18,AIK AMPAT,3,SASAK,1,5


#### Bawak Gunung

In [43]:
# --- BAWAK GUNUNG ---
path_bawak_gunung = 'dataset/bawak-gunung.xlsx'
if os.path.exists(path_bawak_gunung):
    df_bawak_gunung = transform(path_bawak_gunung)
    print(f"\n✅ Selesai! Berhasil memproses {len(df_bawak_gunung)} jiwa di Dusun Bawak Gunung.")

    save_path = os.path.join(output_folder, 'BAWAK_GUNUNG.csv')
    df_bawak_gunung.to_csv(save_path, index=False)
    print(f"💾 File berhasil disimpan di: {save_path}")

else:
    print(f"❌ File {path_bawak_gunung} tidak ditemukan!")

📖 Membuka file: dataset/bawak-gunung.xlsx
   ∟ Memproses RT 01...
   ∟ Memproses RT 02...
   ∟ Memproses RT 03...
   ∟ Memproses RT 04...

✅ Selesai! Berhasil memproses 1030 jiwa di Dusun Bawak Gunung.
💾 File berhasil disimpan di: clean\BAWAK_GUNUNG.csv


In [44]:
clean_bawak_gunung = pd.read_csv('clean/BAWAK_GUNUNG.csv')
clean_bawak_gunung['nik'] = clean_bawak_gunung['nik'].apply(fix_0f_format)
clean_bawak_gunung['no_kk'] = clean_bawak_gunung['no_kk'].apply(fix_0f_format)
print("Sampel Data:")
clean_bawak_gunung[['nama', 'sex_id', 'nik', 'no_kk', 'tanggallahir', 'dusun', 'pekerjaan_id', 'suku', 'agama_id', 'pendidikan_kk_id']].sample(10)

Sampel Data:


,nama,sex_id,nik,no_kk,tanggallahir,dusun,pekerjaan_id,suku,agama_id,pendidikan_kk_id
321,SINALIM,2,5201016112720001,5201011505150003,1958-12-31,BAWAK GUNUNG,19,SASAK,1,3
394,NURHIDAYAH,2,5201014706860004,5201010503088551,1986-05-07,BAWAK GUNUNG,2,SASAK,1,1
766,MUSTAJAB,1,5201013112800300,5201012202180017,1980-12-31,BAWAK GUNUNG,99,SASAK,1,1
647,SANISAH,1,5201013112560055,5201012604120018,1955-12-31,BAWAK GUNUNG,19,SASAK,1,1
297,MURNIATI,2,5201016508820002,5201011808150004,1982-08-25,BAWAK GUNUNG,2,SASAK,1,1
298,RAHUL HUDA,1,5201010504020002,5201011808150004,2002-04-05,BAWAK GUNUNG,99,SASAK,1,3
742,IMRAN,1,5201011107790001,5201011512160007,1977-12-31,BAWAK GUNUNG,19,SASAK,1,1
658,SAMIUN,1,5201013112570008,5201013011690008,1957-12-31,BAWAK GUNUNG,19,NAN,1,3
10,MUHAMAD HAEKAM,1,5201010307150002,5201012210120017,2015-07-03,BAWAK GUNUNG,1,SASAK,1,1
813,JAMIL,1,5201010107510028,5201010503088204,1955-07-01,BAWAK GUNUNG,9,SASAK,1,1


#### Cemare Tengah

In [45]:
# --- CEMARE TENGAH ---
path_cemare_tengah = 'dataset/cemare-tengah.xlsx'
if os.path.exists(path_cemare_tengah):
    df_cemare_tengah = transform(path_cemare_tengah)
    print(f"\n✅ Selesai! Berhasil memproses {len(df_cemare_tengah)} jiwa di Dusun Cemare Tengah.")
    save_path = os.path.join(output_folder, 'CEMARE_TENGAH.csv')
    df_cemare_tengah.to_csv(save_path, index=False)
    print(f"💾 File berhasil disimpan di: {save_path}")

else:
    print(f"❌ File {path_cemare_tengah} tidak ditemukan!")

📖 Membuka file: dataset/cemare-tengah.xlsx
   ∟ Memproses RT 01...
   ∟ Memproses RT 02...
   ∟ Memproses RT 03...

✅ Selesai! Berhasil memproses 869 jiwa di Dusun Cemare Tengah.
💾 File berhasil disimpan di: clean\CEMARE_TENGAH.csv


In [46]:
clean_cemare_tengah = pd.read_csv('clean/CEMARE_TENGAH.csv')
clean_cemare_tengah['nik'] = clean_cemare_tengah['nik'].apply(fix_0f_format)
clean_cemare_tengah['no_kk'] = clean_cemare_tengah['no_kk'].apply(fix_0f_format)
print("Sampel Data:")
clean_cemare_tengah[['nama', 'sex_id', 'nik', 'no_kk', 'tanggallahir', 'dusun', 'pekerjaan_id', 'suku', 'agama_id', 'pendidikan_kk_id']].sample(10)

Sampel Data:


,nama,sex_id,nik,no_kk,tanggallahir,dusun,pekerjaan_id,suku,agama_id,pendidikan_kk_id
115,LIAYU,2,5201015403020002,5201010603089757,2002-05-06,CEMARA,99,NAN,1,1
377,KAMARUDIN,1,5201013112680005,5201011211090017,1958-12-31,CEMARA,88,SASAK,1,3
425,ARINI,2,5201014202110004,5201011409120025,2011-02-02,CEMARA,1,SASAK,1,1
125,TIMAH,2,5201014107720492,5201010202100028,1972-07-01,CEMARA,99,NAN,1,1
635,ANI,2,0,5201011407110001,1990-12-31,CEMARA,99,NAN,1,1
659,KUDIN,1,5201010107670305,5201010503081876,1967-07-01,CEMARA,13,SASAK,1,1
464,ROBY INDRAWAN,1,5201011011970004,5201010404120029,2011-05-10,CEMARA,99,NAN,1,1
82,HAJI HERRY ZOHRI,1,5201010108880307,5201010811160027,1982-11-29,CEMARA,99,NAN,1,1
810,ROHANA S.PD,2,5201015010820005,5201011508160011,1981-10-10,CEMARA,99,NAN,1,1
790,NADIA WAFIDATUZZAHRA,2,5201014107070280,5201011508160011,2008-04-09,CEMARA,99,NAN,1,1


#### Cemare Timur

In [47]:
# --- CEMARE TIMUR ---
path_cemare_timur = 'dataset/cemare-timur.xlsx'
if os.path.exists(path_cemare_timur):
    df_cemare_timur = transform(path_cemare_timur)
    print(f"\n✅ Selesai! Berhasil memproses {len(df_cemare_timur)} jiwa di Dusun Cemare Timur.")
    save_path = os.path.join(output_folder, 'CEMARE_TIMUR.csv')
    df_cemare_timur.to_csv(save_path, index=False)
    print(f"💾 File berhasil disimpan di: {save_path}")

else:
    print(f"❌ File {path_cemare_timur} tidak ditemukan!")

📖 Membuka file: dataset/cemare-timur.xlsx
   ∟ Memproses RT 04...
   ∟ Memproses RT05...

✅ Selesai! Berhasil memproses 426 jiwa di Dusun Cemare Timur.
💾 File berhasil disimpan di: clean\CEMARE_TIMUR.csv


In [48]:
clean_cemare_timur = pd.read_csv('clean/CEMARE_TIMUR.csv')
clean_cemare_timur['nik'] = clean_cemare_timur['nik'].apply(fix_0f_format)
clean_cemare_timur['no_kk'] = clean_cemare_timur['no_kk'].apply(fix_0f_format)
print("Sampel Data:")
clean_cemare_timur[['nama', 'sex_id', 'nik', 'no_kk', 'tanggallahir', 'dusun', 'pekerjaan_id', 'suku', 'agama_id', 'pendidikan_kk_id']].sample(10)

Sampel Data:


,nama,sex_id,nik,no_kk,tanggallahir,dusun,pekerjaan_id,suku,agama_id,pendidikan_kk_id
241,MUHAMMAD TAQIYUDDIN,1,5201010107900106,5201010503083644,1990-01-30,CEMARA TIMUR,3,SASAK,1,5
114,RENAH,1,5201013112640079,5201010504120022,1964-12-31,CEMARA TIMUR,9,SASAK,1,3
342,BUDIMAN,1,5201010206920002,5201010504110032,1992-06-02,CEMARA TIMUR,88,SASAK,1,4
305,SANISAH,2,5201014107650340,5201010503085687,1965-07-01,CEMARA TIMUR,19,SASAK,1,1
192,NURIAH,2,5201015003600001,5201010503085466,1960-03-10,CEMARA TIMUR,20,SASAK,1,3
403,MAHUNI,2,5201015011860002,5201011502120051,1986-11-10,CEMARA TIMUR,2,SASAK,1,1
316,MAFTUH IHSAN,1,5201011601050001,5201010503085850,2005-01-16,CEMARA TIMUR,1,SASAK,1,3
177,SALIKIN,1,5201013112820336,5201011003150004,1982-12-31,CEMARA TIMUR,88,SASAK,1,3
377,H.LUKMAN HAKIM,1,5201013112670100,5201010604120009,1967-12-31,CEMARA TIMUR,88,SASAK,1,3
86,RAKMAN,2,5201016510650002,5201012803120001,1965-10-25,CEMARA TIMUR,99,NAN,1,1


#### Menang Timur

In [49]:
# --- MENANG TIMUR ---
path_menang_timur = 'dataset/menang-timur.xlsx'
if os.path.exists(path_menang_timur):
    df_menang_timur = transform(path_menang_timur)
    print(f"\n✅ Selesai! Berhasil memproses {len(df_menang_timur)} jiwa di Dusun Menang Timur.")
    save_path = os.path.join(output_folder, 'MENANG_TIMUR.csv')
    df_menang_timur.to_csv(save_path, index=False)
    print(f"💾 File berhasil disimpan di: {save_path}")

else:
    print(f"❌ File {path_menang_timur} tidak ditemukan!")

📖 Membuka file: dataset/menang-timur.xlsx
   ∟ Memproses RT 01...
   ∟ Memproses RT 02...
   ∟ Memproses RT 03...

✅ Selesai! Berhasil memproses 255 jiwa di Dusun Menang Timur.
💾 File berhasil disimpan di: clean\MENANG_TIMUR.csv


C:\Users\LEGION\AppData\Local\Temp\ipykernel_48032\3581078191.py:52: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  formatted_dates = pd.to_datetime(raw_dates, errors='coerce')


In [50]:
clean_menang_timur = pd.read_csv('clean/MENANG_TIMUR.csv')
clean_menang_timur['nik'] = clean_menang_timur['nik'].apply(fix_0f_format)
clean_menang_timur['no_kk'] = clean_menang_timur['no_kk'].apply(fix_0f_format)
print("Sampel Data:")
clean_menang_timur[['nama', 'sex_id', 'nik', 'no_kk', 'tanggallahir', 'dusun', 'pekerjaan_id', 'suku', 'agama_id', 'pendidikan_kk_id']].sample(10)

Sampel Data:


,nama,sex_id,nik,no_kk,tanggallahir,dusun,pekerjaan_id,suku,agama_id,pendidikan_kk_id
178,MURAD,1,5201010905700001,5201010503086626,1970-05-09,MENANG TIMUR,19,SASAK,1,3
229,SITI MELINA,2,5201014403080005,5201010503086614,2019-03-24,MENANG TIMUR,99,NAN,1,1
215,MUSLEH UDIN,1,5201011906850001,5201011304120026,1985-06-19,MENANG TIMUR,88,SASAK,1,5
213,M. ABDUROSYID,1,5201013107160001,5201010610140008,2019-07-31,MENANG TIMUR,99,NAN,1,1
208,M ALI,1,5201011011830005,5201012709160023,2019-11-10,MENANG TIMUR,19,NAN,1,1
201,MUSTIANI,2,5201014202991001,5201013108160002,2019-02-02,MENANG TIMUR,99,NAN,1,1
211,NASIP,1,5201010211800001,5201010610140008,2019-11-02,MENANG TIMUR,88,NAN,1,1
157,MUSAHAB,1,5201011111060001,5201010503086970,NaN,MENANG TIMUR,19,SASAK,1,1
251,SAIPUL BAHRI,1,0,5201012405120018,1970-07-01,MENANG TIMUR,6,NAN,1,1
81,TAHMIN,1,5201010107670677,5201010503086825,NaN,MENANG TIMUR,99,SASAK,1,1


### Formatting (BTN-PEMDA)

In [51]:
output_folder = 'clean'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    print(f"📁 Folder '{output_folder}' berhasil dibuat.")

def transform_btn(file_path):
    print(f"📖 Membuka file: {file_path}")
    
    xls = pd.ExcelFile(file_path)
    all_rt_data = []

    for sheet in xls.sheet_names:
        if "RT" not in sheet.upper(): continue
        print(f"   ∟ Memproses {sheet}...")
        
        # skiprows=4 karena header utama ada di baris 1-4
        df = pd.read_excel(xls, sheet_name=sheet, skiprows=4)
        if df.empty or len(df.columns) < 10: continue
        
        # Melewati sub-header (Tempat, Tanggal, Tahun) yang biasanya ada di baris pertama data
        df_clean = df.iloc[2:].reset_index(drop=True)
        
        # Forward Fill No. KK (Kolom index 2) - Mengatasi sel kosong di bawah KK
        df_clean.iloc[:, 2] = df_clean.iloc[:, 2].replace(0, np.nan).ffill()

        new_df = pd.DataFrame(index=df_clean.index, columns=target_cols)
        
        header_df = pd.read_excel(xls, sheet_name=sheet, nrows=5, header=None)
        raw_dusun = str(header_df.iloc[2, 1]) # Biasanya di baris 3 (index 2), kolom B (index 1)
        
        # Regex untuk hapus "Lingkungan" atau kata pertama (case insensitive)
        clean_dusun = re.sub(r'^(lingkungan|dusun)\s+', '', raw_dusun, flags=re.IGNORECASE).strip().upper()
        
        # 1. Mapping Identitas (Menggunakan index kolom agar stabil)
        new_df['alamat'] = df_clean.iloc[:, 11].astype(str).str.upper().str.strip()
        new_df['nama'] = df_clean.iloc[:, 3].astype(str).str.upper().str.strip()
        # new_df['no_kk'] = df_clean.iloc[:, 2].astype(str).str.replace(".0", "", regex=False).str.replace(" ", "")
        new_df['nik'] = df_clean.iloc[:, 13].astype(str).str.replace(".0", "", regex=False).str.replace(" ", "")
        new_df['sex'] = df_clean.iloc[:, 4].astype(str).str.upper().str.strip()
        new_df['status_kawin'] = df_clean.iloc[:, 5].astype(str).str.upper().str.strip()
        new_df['dusun'] = "BTN Pemda"
        new_df['rt'] = sheet.replace("RT", "").strip()
        # new_df['tempatlahir'] = df_clean.iloc[:, 9].astype(str).str.upper()
        # new_df['suku'] = df_clean.iloc[:, 14].astype(str).str.upper().str.strip()
        new_df['pekerjaan_id'] = df_clean.iloc[:, 9].astype(str).str.upper().str.strip()
        new_df['agama_id'] = df_clean.iloc[:, 7].astype(str).str.upper().str.strip()
        new_df['kk_level_id'] = df_clean.iloc[:, 12].astype(str).str.upper().str.strip()
        new_df['pendidikan_kk_id'] = df_clean.iloc[:, 8].astype(str).str.upper().str.strip()
        new_df['warganegara_id'] = df_clean.iloc[:, 10].astype(str).str.upper().str.strip()
        # 2. Transformasi Tanggal
        raw_dates = df_clean.iloc[:, 6]
        formatted_dates = pd.to_datetime(raw_dates, format='%m/%d/%y', errors='coerce')
        new_df['tanggallahir'] = formatted_dates.dt.strftime('%Y-%m-%d')
        
        # 3. Mapping ID Numerik
        new_df['sex_id'] = df_clean.iloc[:, 4].apply(lambda x: clean_and_map(x, map_sex, 1))
        new_df['status_kawin_id'] = df_clean.iloc[:, 5].apply(lambda x: clean_and_map(x, map_kawin, 1))
        new_df['pekerjaan_id'] = df_clean.iloc[:, 9].apply(lambda x: clean_and_map(x, map_pekerjaan, 99))
        new_df['agama_id'] = df_clean.iloc[:, 7].apply(lambda x: clean_and_map(x, map_agama, 1))
        new_df['kk_level_id'] = df_clean.iloc[:, 12].apply(lambda x: clean_and_map(x, map_kk_level, 1))
        new_df['pendidikan_kk_id'] = df_clean.iloc[:, 8].apply(lambda x: clean_and_map(x, map_pendidikan_kk, 1))
        new_df['warganegara_id'] = df_clean.iloc[:, 10].apply(lambda x: clean_and_map(x, map_warganegara, 1))
        
        # 4. Default Values
        # new_df[['status_dasar', 'warganegara_id']] = 1
        # new_df[['cacat_id', 'hamil']] = [7, 2]
        # new_df['suku'] = 'SASAK'
        
        all_rt_data.append(new_df)

    if all_rt_data:
        final_df = pd.concat(all_rt_data, ignore_index=True)
        # Hapus baris yang namanya kosong/nan
        return final_df[final_df['nama'].notna() & (final_df['nama'] != 'NAN')]
    else:
        return pd.DataFrame(columns=target_cols)

#### BTN Pemda

In [52]:
# --- BTN PEMDA ---
path_btn_pemda = 'dataset/btn-pemda.xlsx'
if os.path.exists(path_btn_pemda):
    df_btn_pemda = transform_btn(path_btn_pemda)
    print(f"\n✅ Selesai! Berhasil memproses {len(df_btn_pemda)} jiwa di Dusun Menang Timur.")
    save_path = os.path.join(output_folder, 'BTN_PEMDA.csv')
    df_btn_pemda.to_csv(save_path, index=False)
    print(f"💾 File berhasil disimpan di: {save_path}")

else:
    print(f"❌ File {path_btn_pemda} tidak ditemukan!")

📖 Membuka file: dataset/btn-pemda.xlsx
   ∟ Memproses RT 01...
   ∟ Memproses RT 02...
   ∟ Memproses RT 03...
   ∟ Memproses RT 04...
   ∟ Memproses RT 05...

✅ Selesai! Berhasil memproses 1285 jiwa di Dusun Menang Timur.
💾 File berhasil disimpan di: clean\BTN_PEMDA.csv


In [53]:
clean_btn_pemda = pd.read_csv('clean/BTN_PEMDA.csv')
clean_btn_pemda['nik'] = clean_btn_pemda['nik'].apply(fix_0f_format)
clean_btn_pemda['no_kk'] = clean_btn_pemda['no_kk'].apply(fix_0f_format)
print("Sampel Data:")
clean_btn_pemda[['nama', 'sex_id', 'nik', 'no_kk', 'tanggallahir', 'dusun', 'pekerjaan_id', 'suku', 'agama_id', 'pendidikan_kk_id']].sample(10)

Sampel Data:


,nama,sex_id,nik,no_kk,tanggallahir,dusun,pekerjaan_id,suku,agama_id,pendidikan_kk_id
771,PURWA SAPUTRA,2,5201010306020001,,2002-06-03,BTN Pemda,99,NaN,1,1
1030,BAIQ LUNA ZAHRA,2,5201018710070001,,2007-10-27,BTN Pemda,99,NaN,1,1
481,JOHENDI FIRMAN MAAKH,1,5201010203850003,,1986-03-02,BTN Pemda,6,NaN,2,1
452,LALU RAYSARABRAR,1,5201010604090001,,2009-04-06,BTN Pemda,99,NaN,1,1
654,JESSY WAHYUNING TYAS,2,0,,2015-04-16,BTN Pemda,99,NaN,1,1
822,ENDAH SUSANTI,2,520101200403047,,1980-11-27,BTN Pemda,6,NaN,1,1
473,NI MADE PRADNYA WIRIANI,2,5201014107930163,,1993-02-10,BTN Pemda,99,NaN,4,1
1040,MARNAH MUNIR,2,5201018203720002,,1972-03-12,BTN Pemda,6,NaN,1,1
443,RIVA YUNI LESTARI,2,5201014906930001,,1993-06-09,BTN Pemda,99,NaN,1,1
60,AYUNDITA MAHARANI SETIAWAN,2,5201015712040002,,2004-12-17,BTN Pemda,99,NaN,1,1


### Data Clean

In [3]:
import glob

path_clean = "clean"
all_files = glob.glob(os.path.join(path_clean, "*.csv"))

dfs = []
for file in all_files:
    df = pd.read_csv(file)

    # tambahin kolom sumber dataset (nama file)
    nama_file = os.path.basename(file).replace(".csv", "")
    df["Kelurahan-Dasan-Geres"] = nama_file

    dfs.append(df)

# gabungkan semua dataframe
df_all = pd.concat(dfs, ignore_index=True)

print("Total data:", df_all.shape)
df_all.info()

Total data: (8530, 48)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8530 entries, 0 to 8529
Data columns (total 48 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   alamat                 1285 non-null   object 
 1   dusun                  8530 non-null   object 
 2   rw                     0 non-null      float64
 3   rt                     8530 non-null   int64  
 4   nama                   8530 non-null   object 
 5   no_kk                  7123 non-null   float64
 6   nik                    8391 non-null   object 
 7   sex                    8530 non-null   object 
 8   tempatlahir            7245 non-null   object 
 9   tanggallahir           8356 non-null   object 
 10  agama_id               8530 non-null   int64  
 11  pendidikan_kk_id       8530 non-null   int64  
 12  pendidikan_sedang_id   0 non-null      float64
 13  pekerjaan_id           8530 non-null   int64  
 14  status_kawin           8530 non-n

In [4]:
df_all.to_csv("Kelurahan-Dasan-Geres.csv", index=False)
print("Berhasil disimpan: Kelurahan-Dasan-Geres.csv")

Berhasil disimpan: Kelurahan-Dasan-Geres.csv


## EDA

In [6]:
df_all["dusun"].value_counts().head(20)


dusun
BTN Pemda             1285
DASAN GERES BARAT     1111
AIK AMPAT             1078
BAWAK GUNUNG          1030
DASAN GERES TENGAH     883
CEMARA                 869
DASAN GERES TIMUR      827
DASN GERES SELATAN     766
CEMARA TIMUR           426
MENANG TIMUR           255
Name: count, dtype: int64

In [7]:
df_all["agama_id"].value_counts().head(10)


agama_id
1    8410
4     107
2      13
Name: count, dtype: int64

In [10]:
df_all["pendidikan_kk_id"].value_counts().head(10)

pendidikan_kk_id
1    5477
3    1223
5     710
4     628
2     430
8      49
7       7
6       6
Name: count, dtype: int64

In [11]:
df_all["pekerjaan_id"].value_counts().head(10)


pekerjaan_id
99    3700
88    1037
19     767
1      696
3      634
2      553
6      379
84     264
9      160
20     125
Name: count, dtype: int64

In [12]:
df_all["kk_level_id"].value_counts().head(10)

kk_level_id
4     3708
1     2670
3     2008
11      44
6       37
7       29
9       28
8        5
2        1
Name: count, dtype: int64

In [13]:
df_all["status_kawin"].value_counts()

status_kawin
KAWIN                   4020
BELUM KAWIN             3822
JANDA                    253
CERAI MATI               198
CERAI HIDUP               88
DUDA                      72
KAWIN BELUM TERCATAT      20
KAWIN TERCATAT            10
NAN                        6
BELUM  KAWIN               5
BLUM KAWIN                 4
CERAI                      4
BELUMKAWIN                 3
KAWN                       2
-                          2
KAWAN                      1
BELEM KAWIN                1
BELUM KAWIN-               1
KAWWIN                     1
KAIN                       1
BELUM KAWIN `              1
KAWIU                      1
BELUM/KAWIN                1
BELUM KAWI                 1
KAWI                       1
BELUM KAKWIN               1
KWIN                       1
BELUM KAWAN                1
KAWAIN                     1
BELUM KAEIN                1
BELUM                      1
BELUM KAIN                 1
BWLUM KAWIN                1
ANAK                       1
K

In [14]:
def clean_status_kawin(x):
    if pd.isna(x):
        return "TIDAK DIKETAHUI"
        
    x = str(x).strip().upper()

    # buang karakter aneh
    x = x.replace("`", "").replace("-", " ").strip()

    # normalisasi typo umum
    x = x.replace("BELUM  KAWIN", "BELUM KAWIN")
    x = x.replace("BELUMKAWIN", "BELUM KAWIN")
    x = x.replace("BLUM KAWIN", "BELUM KAWIN")
    x = x.replace("BELEM KAWIN", "BELUM KAWIN")
    x = x.replace("BEKUM KAWIN", "BELUM KAWIN")
    x = x.replace("BWLUM KAWIN", "BELUM KAWIN")
    x = x.replace("KAWN", "KAWIN")
    x = x.replace("KWIN", "KAWIN")
    x = x.replace("KAWWIN", "KAWIN")
    x = x.replace("KAWIU", "KAWIN")
    x = x.replace("KAWI", "KAWIN")
    x = x.replace("KAIN", "KAWIN")
    x = x.replace("KAWAIN", "KAWIN")
    x = x.replace("KAWAN", "KAWIN")
    x = x.replace("KAWIN KAWIN", "KAWIN")

    # aturan grouping final
    if "BELUM" in x:
        return "BELUM KAWIN"
    if "KAWIN" in x:
        return "KAWIN"
    if "CERAI HIDUP" in x:
        return "CERAI HIDUP"
    if "CERAI MATI" in x:
        return "CERAI MATI"
    if x in ["JANDA", "DUDA"]:
        return "JANDA/DUDA"
    if "CERAI" in x:
        # kalau cuma "CERAI" tanpa jelas, kita taruh CERAI HIDUP (bisa diubah)
        return "CERAI HIDUP"
    if x in ["", "NAN", "NONE"]:
        return "TIDAK DIKETAHUI"
    return "LAINNYA"

df_all["status_kawin_clean"] = df_all["status_kawin"].apply(clean_status_kawin)

df_all["status_kawin_clean"].value_counts()


status_kawin_clean
KAWIN              4040
BELUM KAWIN        3866
JANDA/DUDA          325
CERAI MATI          198
CERAI HIDUP          92
TIDAK DIKETAHUI       8
LAINNYA               1
Name: count, dtype: int64

In [15]:
def clean_dusun(x):
    if pd.isna(x):
        return "TIDAK DIKETAHUI"
    x = str(x).strip().upper()
    x = x.replace("DASN GERES", "DASAN GERES")
    x = x.replace("CEMARE", "CEMARA")
    return x

df_all["dusun_clean"] = df_all["dusun"].apply(clean_dusun)
df_all["dusun_clean"].value_counts()


dusun_clean
BTN PEMDA              1285
DASAN GERES BARAT      1111
AIK AMPAT              1078
BAWAK GUNUNG           1030
DASAN GERES TENGAH      883
CEMARA                  869
DASAN GERES TIMUR       827
DASAN GERES SELATAN     766
CEMARA TIMUR            426
MENANG TIMUR            255
Name: count, dtype: int64

In [16]:
df_all["tanggallahir"] = pd.to_datetime(df_all["tanggallahir"], errors="coerce")

today = pd.Timestamp.today()
df_all["umur"] = (today - df_all["tanggallahir"]).dt.days // 365

def kelompok_umur(u):
    if pd.isna(u):
        return "Tidak diketahui"
    u = int(u)
    if u <= 5: return "0-5"
    if u <= 12: return "6-12"
    if u <= 17: return "13-17"
    if u <= 25: return "18-25"
    if u <= 40: return "26-40"
    if u <= 60: return "41-60"
    return "60+"

df_all["kelompok_umur"] = df_all["umur"].apply(kelompok_umur)
df_all["kelompok_umur"].value_counts()


kelompok_umur
41-60              2606
26-40              2042
18-25              1072
60+                1048
13-17               828
6-12                758
Tidak diketahui     174
0-5                   2
Name: count, dtype: int64

In [17]:
df_all.to_csv("Kelurahan-Dasan-Geres-Fix.csv", index=False)
print("Berhasil disimpan: Kelurahan-Dasan-Geres-Fix.csv")

Berhasil disimpan: Kelurahan-Dasan-Geres-Fix.csv


In [9]:

# def clean_and_map(val, mapping, default=99):
#     if pd.isna(val) or val == "" or val == "-": return default
#     v = str(val).strip().upper()
#     # Mencari kunci yang terkandung dalam teks (fuzzy matching sederhana)
#     for key in mapping:
#         if key in v:
#             return mapping[key]
#     return default

# def process_sid_file(file_path, dusun_name):
#     print(f"Sedang memproses Dusun: {dusun_name}")
#     xls = pd.ExcelFile(file_path)
#     combined_rt = []
#     for sheet in xls.sheet_names:
#         if "RT" not in sheet.upper(): continue
#         df_raw = pd.read_excel(xls, sheet_name=sheet, skiprows=4)
#         df_raw.columns = [str(c).strip() for c in df_raw.columns]
        
#         new_df = pd.DataFrame(columns=target_columns)
#         new_df['nama'] = df_raw.get('Nama Lengkap/ Panggilan', df_raw.get('Nama', '')).astype(str).str.upper()
#         new_df['pekerjaan_id'] = df_raw.get('Pekerjaan', '').apply(lambda x: clean_and_map(x, map_pekerjaan, 99))
#         new_df['dusun'] = dusun_name.upper()
#         new_df['rt'] = sheet.replace("RT", "").strip()
#         new_df['status_dasar'] = 1 # HIDUP (Default)
        
#         # Mapping kolom identitas dasar lainnya
#         new_df['sex'] = df_raw.get('Jenis Kelamin (L/P)', '').apply(lambda x: clean_and_map(x, map_sex, 1))
#         new_df['agama_id'] = df_raw.get('Agama', '').apply(lambda x: clean_and_map(x, map_agama, 1))
        
#         combined_rt.append(new_df)
#     return pd.concat(combined_rt, ignore_index=True)

In [56]:
# # 3. DAFTAR FILE DUSUN (2 AKTIF, 8 COMMENTED)
# files_list = [
#     ('btn-pemda.xlsx', 'PERUMDA SELATAN'),
#     ('dasan-geres-tengah.xlsx', 'DASAN GERES TENGAH'),
#     # ('aik-ampat.xlsx', 'AIK AMPAT'),
#     # ('bawak-gunung.xlsx', 'BAWAK GUNUNG'),
#     # ('cemare-tengah.xlsx', 'CEMARE TENGAH'),
#     # ('cemare-timur.xlsx', 'CEMARE TIMUR'),
#     # ('dasan-geres-barat.xlsx', 'DASAN GERES BARAT'),
#     # ('dasan-geres-selatan.xlsx', 'DASAN GERES SELATAN'),
#     # ('dasan-geres-timur.xlsx', 'DASAN GERES TIMUR'),
#     # ('menang-timur.xlsx', 'MENANG TIMUR'),
# ]

# # 4. EKSEKUSI PENGGABUNGAN DATA
# all_data = []
# for file, dusun in files_list:
#     if os.path.exists(file):
#         all_data.append(process_sid_file(file, dusun))

# if all_data:
#     final_master = pd.concat(all_data, ignore_index=True)
#     final_master.dropna(subset=['nama'], inplace=True)
    
#     # Simpan Hasil ke CSV
#     final_master.to_csv('MASTER_SID_COMPLETE_99.csv', index=False)
#     print(f"\nSukses! {len(final_master)} baris data telah dipetakan dengan 99 Kode Pekerjaan.")
#     print("File tersimpan sebagai: MASTER_SID_COMPLETE_99.csv")